<a href="https://colab.research.google.com/github/acb100cias/BioMateFC/blob/master/FN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.integrate import odeint

In [ ]:
P=[{'a':-0.3,'b':1.4,'tau':20,'I':0},
 {'a':-0.3,'b':1.4,'tau':20,'I':0.23},
   {'a':-0.3,'b':1.4,'tau':20,'I':0.5}]

T=np.linspace(0,500,1500)

In [ ]:
def FN(x,t,a,b,tau,I):
  return  np.array([x[0]-x[0]**3-x[1]+I,
                    (x[0]-a-b*x[1])/tau])


In [ ]:
v,w=odeint(FN,np.array([0,0]),T,args=(P[0]['a'],P[0]['b'],
                            P[0]['tau'],P[0]['I'])).T

In [ ]:
v1,w1=odeint(FN,np.array([0,0]),T,args=(P[1]['a'],P[1]['b'],
                            P[1]['tau'],P[1]['I'])).T

In [ ]:
v2,w2=odeint(FN,np.array([0,0]),T,args=(P[2]['a'],P[2]['b'],
                            P[2]['tau'],P[2]['I'])).T

In [ ]:
fig,ax=plt.subplots(1,3,figsize=(10,5))
ax[0].plot(T,v)
ax[0].plot(T,w)
ax[1].plot(T,v1)
ax[1].plot(T,w1)
ax[2].plot(T,v2)
ax[2].plot(T,w2)

In [ ]:
from matplotlib import streamplot

In [ ]:
x=np.linspace(-1,1,100)
y=np.array([(1/P[0]['b'])*(k-P[0]['a']) for k in x])
X,Y=np.meshgrid(x,y)

In [ ]:
y

In [ ]:
fig,ax=plt.subplots(1,3,figsize=(10,5))
dx,dy=FN(np.array([X,Y]),0,P[0]['a'],P[0]['b'],
         P[0]['tau'],P[0]['I'])
ax[0].streamplot(X,Y,dx,dy,color='k')

dx1,dy1=FN(np.array([X,Y]),0,P[1]['a'],P[1]['b'],
         P[1]['tau'],P[1]['I'])
ax[1].streamplot(X,Y,dx1,dy1,color='k')


dx2,dy2=FN(np.array([X,Y]),0,P[2]['a'],P[2]['b'],
         P[2]['tau'],P[2]['I'])
ax[2].streamplot(X,Y,dx2,dy2,color='k')

In [ ]:
def find_roots(a,b,I,tau):
  coef=[1,0,1/b-1,-a/b-I]
  roots=[np.real(r) for r in np.roots(coef)
  if np.isreal(r)]
  return [[r,r-r**3+I] for r in roots]
eqnproot={}
for i,p in enumerate(P):
  eqnproot[i]=find_roots(**p)

In [ ]:
eqnproot

In [ ]:
EQUILIBRIUM_COLOR={'Stable node':'C0',
                   'Unstable node':'C1',
                   'Saddle':'C4',
                   'Stable focus':'C3',
                   'Unstable focus':'C2',
                   'Center': 'C5'
                   }

In [ ]:
def jacobiano_FN(v,w,a,b,tau,I):
  return([[-3*v**2+1,-1],[1/tau,-b/tau]])

def estabilidad(jacobiano):
  ev=np.linalg.eigvals(jacobiano)
  if all(np.real(ev)==0) and all(np.imag(ev)!=0):
    nature='Center'
  elif np.real(ev)[0]*np.real(ev)[1]<0:
    nature='Saddle'
  else:
    stability ='Unstable' if all (np.real(ev)>0) else 'Stable'
    nature=stability+(' focus' if all(np.imag(ev)!=0) else ' node')
  return nature




In [ ]:
ispan=np.linspace(0,0.5,200)
bspan=np.linspace(0.6,2,200)

In [ ]:
I_list=[]
eqs_list=[]
nature_legends=[]
trace=[]
det=[]

In [ ]:
for I in ispan:
  param={'I':I,'a':-0.3,'b':1.4,'tau':20}
  roots=find_roots(**param)
  for v,w in roots:
    J=jacobiano_FN(v,w,**param)
    nature=estabilidad(J)
    nature_legends.append(nature)
    I_list.append(I)
    eqs_list.append(v)
    det.append(np.linalg.det(J))
    trace.append(np.trace(J))

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(10,5))
labels=frozenset(nature_legends)
ax.scatter(I_list,eqs_list,c=[EQUILIBRIUM_COLOR[l] for l in nature_legends]
           ,s=5.9)
ax.legend([mpatches.Patch(color=EQUILIBRIUM_COLOR[l])for l in labels],
          labels,loc='lower right')
ax.set(xlabel='$I$',ylabel='$v$')
